In [ ]:
# standard imports
import os
import re
import sys
import requests
import unicodedata
import pandas as pd
from bs4 import BeautifulSoup

# natural language processing
import nltk

In [ ]:
# pandas display settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None

In [ ]:
# number of ngrams to consider from each blog
terms_to_consider = 20

In [ ]:
# read and store all stopwords in lowercase
curr_path = os.path.abspath("nouns_from_ngrams.ipynb")
stopwords_path = os.path.abspath(os.path.join(curr_path, "../../../..", "Read_Files", "stopwords_cleaned.txt"))
with open(stopwords_path) as file:
    stopwords = [line.strip().lower() for line in file]

In [ ]:
# get all links
csv_path = os.path.abspath(os.path.join(curr_path, "../..", "alokeveer_analytics_data.csv"))
df = pd.read_csv(csv_path)
no_of_blogs = df.shape[0]

In [ ]:
# lists for storing data, _i stands for i-gram data
successful_blog_list = []
noun_lists_1 = []
noun_lists_2 = []
noun_lists_3 = []
occurence_count_1 = {}
occurence_count_2 = {}
occurence_count_3 = {}

In [ ]:
# function to get all page content from html response
def get_page_text(html_response):
    # getting page content
    html_text = html_response.text
    soup = BeautifulSoup(html_text, "lxml")

    # various sources of text
    para_text = [element.text.strip() for element in soup.find_all("p")]
    header_text = [element.text.strip() for element in soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])]
    span_text = [element.text.strip() for element in soup.find_all("span")]
    all_text = para_text + header_text + span_text
    
    return " ".join(all_text)

In [ ]:
# function to cleanup text and find words in it
def extract_words_from_text(text):
    # text cleaning
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

    # replace '/' and '-' in words for correct spacing
    text = text.replace("/", " ")
    text = text.replace("-", " ")
    words = re.sub(r'[^\w\s]', '', text).split()

    # word list
    return [word for word in words if word.lower() not in stopwords]

In [ ]:
# function to find most frequent ngrams from word list
def find_ngrams(words, n):
    # find term frequency
    series = (pd.Series(nltk.ngrams(words, n)).value_counts())[:terms_to_consider]
    
    # store top results
    ngrams = []
    for tup in series.index:
        ngrams.append(" ".join(tup))
    
    # ensure length of list for appending
    for i in range(max(terms_to_consider - len(ngrams), 0)):
        ngrams.append("")
    
    return ngrams

In [ ]:
# function to return words having capital letters from frequent ngrams
def find_nouns(ngrams):
    nouns = [ngram for ngram in ngrams if ((not ngram.islower()) and (not ngram.isupper()))]
    
    # ensure length of list for appending
    for i in range(max(terms_to_consider - len(nouns), 0)):
        nouns.append("")
    
    return nouns

In [ ]:
# function to find document frequency of each noun
def fill_occurence_dict(noun_lists):
    dictionary = {}
    
    # initialize dictionary
    for noun_list in noun_lists:
        for noun in noun_list:
            if noun != "":
                dictionary[noun] = []

    # fill unique occurence
    for i in range(len(noun_lists)):
        for noun in noun_lists[i]:
            if (noun != "") and (i not in dictionary[noun]):
                dictionary[noun].append(i)

    return dictionary

In [ ]:
# function to print frequently occuring nouns
def print_top_nouns(dictionary):
    # create df from dictionary
    df_top_nouns = pd.DataFrame(columns=["keyword", "occurence"])
    for key, value in dictionary.items():
        if len(value) > 1:
            df_top_nouns.loc[len(df_top_nouns.index)] = [key, len(value)]
    
    # sort based on frequency
    df_top_nouns.sort_values("occurence", ascending=False, inplace=True)
    df_top_nouns.reset_index(inplace=True, drop=True)

    print(df_top_nouns.to_string())
    print()

In [ ]:
# iterate through all blogs
for blog_link in df["Blog Link"]:
    # getting page response
    html_response = requests.get(blog_link)
    if(html_response.status_code != 200):
        continue

    # get page content
    all_text = get_page_text(html_response)

    # get words list
    words = extract_words_from_text(all_text)

    # get top ngrams for each blog
    ngrams_1 = find_ngrams(words, 1)
    ngrams_2 = find_ngrams(words, 2)
    ngrams_3 = find_ngrams(words, 3)
    
    # get nouns from ngrams
    nouns_1 = find_nouns(ngrams_1)
    nouns_2 = find_nouns(ngrams_2)
    nouns_3 = find_nouns(ngrams_3)

    # store data
    successful_blog_list.append(blog_link)
    noun_lists_1.append(nouns_1)
    noun_lists_2.append(nouns_2)
    noun_lists_3.append(nouns_3)

In [ ]:
# fill all occurence dictionaries
occurence_count_1 = fill_occurence_dict(noun_lists_1)
occurence_count_2 = fill_occurence_dict(noun_lists_2)
occurence_count_3 = fill_occurence_dict(noun_lists_3)

In [ ]:
# print all output
# set output path
output_path = os.path.abspath(os.path.join(curr_path, "../..", "Outputs/nouns_from_ngrams.txt"))

# write to file
original_stdout = sys.stdout
with open(output_path, "w") as f:
    sys.stdout = f

    print("KEYWORDS OCCURING IN MULTIPLE BLOGS:")
    print()
    # print summary
    print("1-gram:")    
    print_top_nouns(occurence_count_1)
    print("2-gram:")
    print_top_nouns(occurence_count_2)
    print("3-gram:")
    print_top_nouns(occurence_count_3)

    print()
    print()
    print()

    print("INDIVIDUAL BLOG ANALYSIS:")
    print()

    # print blog wise result
    for i in range(len(successful_blog_list)):    
        print(successful_blog_list[i])

        # create df for output
        df_output = pd.DataFrame()
        df_output["1-gram"] = noun_lists_1[i]
        df_output["2-gram"] = noun_lists_2[i]
        df_output["3-gram"] = noun_lists_3[i]
        
        # remove empty rows
        df_output = df_output.loc[df_output["1-gram"] + df_output["2-gram"] + df_output["3-gram"] != ""]
        
        print(df_output.to_string())
        print()
    
    # reset stdout
    sys.stdout = original_stdout